In [1]:
from scipy.ndimage.morphology import binary_fill_holes, binary_dilation, binary_erosion, generate_binary_structure
from skimage.morphology import label
import nrrd
import numpy as np

## Flood Fill

In [7]:
# vessel, _ = nrrd.read('results/vessel9.nrrd')
# dilated = binary_dilation(vessel)
# kidney, _ = nrrd.read('results/kidney9.nrrd')
# dilated = binary_dilation(kidney)
artery, _ = nrrd.read('results/artery9.nrrd')
dilated = binary_dilation(artery)

In [8]:
X, Y, Z = dilated.shape

In [9]:
x_mat = np.zeros_like(dilated)
y_mat = np.zeros_like(dilated)
z_mat = np.zeros_like(dilated)

Process 3d cube with Flood Fill Algorithm slice by slice

In [10]:
for x in range(X):
    sc = dilated[x,:,:]
    filled_sc = binary_fill_holes(sc)
    x_mat[x,:,:] = filled_sc

In [11]:
for y in range(Y):
    sc = dilated[:,y,:]
    filled_sc = binary_fill_holes(sc)
    y_mat[:,y,:] = filled_sc

In [12]:
for z in range(Z):
    sc = dilated[:,:,z]
    filled_sc = binary_fill_holes(sc)
    z_mat[:,:,z] = filled_sc

Fuse models from 3 directions(x,y,z) with and/or operation

In [13]:
or_mat = np.logical_or(np.logical_or(x_mat, y_mat), z_mat)
nrrd.write('results/2d_fill_or_artery9.nrrd', or_mat.astype(np.float))
and_mat = np.logical_and(np.logical_and(x_mat, y_mat), z_mat)
nrrd.write('results/2d_fill_and_artery9.nrrd', and_mat.astype(np.float))

# or_mat = np.logical_or(np.logical_or(x_mat, y_mat), z_mat)
# nrrd.write('results/2d_fill_or_kidney9.nrrd', or_mat.astype(np.float))
# and_mat = np.logical_and(np.logical_and(x_mat, y_mat), z_mat)
# nrrd.write('results/2d_fill_and_kidney9.nrrd', and_mat.astype(np.float))

In [3]:
# get ablation
kidney6, _ = nrrd.read('results/2d_fill_or_kidney6.nrrd')
kidney9, _ = nrrd.read('results/2d_fill_or_kidney9.nrrd')
ablation_kidney = kidney9 - kidney6
ablation_kidney[np.where(ablation_kidney<0)] = 0
nrrd.write('results/ablation_kidney.nrrd', ablation_kidney.astype(np.float))

In [2]:
vessel9, _ = nrrd.read('results/vessel9.nrrd')
kidney9, _ = nrrd.read('results/2d_fill_or_kidney9.nrrd')

In [3]:
# add erosion on kidney to get larger mask
kidney9 = binarall_masky_dilation(kidney9)
ablation_kidney = (1-kidney9)*vessel9
nrrd.write('results/ablation_kidney.nrrd', ablation_kidney.astype(np.float))

In [4]:
artery, _ = nrrd.read('results/2d_fill_or_artery9.nrrd')

In [6]:
artery = binary_dilation(artery)
ablation_kidney_artery = (1-artery)*ablation_kidney
nrrd.write('results/ablation_kidney_artery.nrrd', ablation_kidney_artery.astype(np.float))

## Get tiny vessel

In [2]:
ablation_kidney_artery, _ = nrrd.read('results/ablation_kidney_artery.nrrd')
new_mask, num = label(ablation_kidney_artery, neighbors=4, background=None, return_num=True, connectivity=None)
order_ind = list()
for i in range(num):
    inds = np.where(new_mask==i)
    mask_size = inds[0].shape[0]
    if mask_size > 50:
        order_ind.append((i, mask_size))

In [2]:
all_mask = np.zeros(new_mask.shape)
if len(order_ind)>1:
    sorted_ind = sorted(order_ind, key=lambda x: x[1], reverse=True)
    selected_inds = sorted_ind[4:]
    for j in selected_inds:
        inds = np.where(new_mask==j[0])
        print(inds[0].shape)
        all_mask[inds] = 1

In [5]:
nrrd.write('results/tiny_vessel.nrrd', all_mask.astype(np.float))